In [1]:
import csv

# baca data csv , simpan ke bentuk list
with open("Fitur/area.csv") as f:
    reader = csv.reader(f)
    # fungsi list() untuk mengubah ke bentuk list()
    area = list(reader)[0]
    
with open("Fitur/eccentricity.csv") as f:
    reader = csv.reader(f)
    eccentricity = list(reader)[0]

with open("Fitur/perimeter.csv") as f:
    reader = csv.reader(f)
    perimeter = list(reader)[0]
    
with open("Fitur/metric.csv") as f:
    reader = csv.reader(f)
    metric = list(reader)[0]
    
with open("Fitur/red.csv") as f:
    reader = csv.reader(f)
    red = list(reader)[0]
    
with open("Fitur/green.csv") as f:
    reader = csv.reader(f)
    green = list(reader)[0]
    
with open("Fitur/blue.csv") as f:
    reader = csv.reader(f)
    blue = list(reader)[0]

with open("Fitur/blue.csv") as f:
    reader = csv.reader(f)
    blue = list(reader)[0]
    
with open("Fitur/label.csv") as f:
    reader = csv.reader(f)
    label = list(reader)[0]
    
# ganti label dalam bentuk string
idx = 0
for i in label:
    if i == "1":
        label[idx] = "Cocor Bebek"
    elif i == "2":
        label[idx] = "Jambu"
    elif i == "3":
        label[idx] = "Kacang Panjang"
    elif i == "4":
        label[idx] = "Singkong"
    elif i == "5":
        label[idx] = "Tomat"
        
    idx = idx+1

In [2]:
import pandas as pd
# fungsi map() untuk mengubah tipe data semua element dalam list
# fungsi zip() untuk menggandengkan setiap list menjadi matriks
df = pd.DataFrame(list(zip(list(map(int,area)), list(map(float,eccentricity)), list(map(float,perimeter)), list(map(float,metric)), list(map(float,red)), list(map(float,green)), list(map(float,blue)), label)), columns = ['Area', 'Eccentricity', 'Perimeter', 'Metric', 'Red', 'Green', 'Blue', 'Label'])

In [3]:
# melihat 5 data terakhir, sekedar melihat struktur tabel
df.tail()

,Area,Eccentricity,Perimeter,Metric,Red,Green,Blue,Label
95,2592,0.587480,648.558,0.077437,62.147971,98.496181,34.956567,Tomat
96,2034,0.744930,296.505,0.290735,48.569492,67.977778,38.859887,Tomat
97,1796,0.736844,262.322,0.327980,82.097821,107.617685,66.557027,Tomat
98,1931,0.789033,542.651,0.082404,62.952130,88.649493,54.805680,Tomat
99,1787,0.792165,525.614,0.081283,80.768362,111.433290,70.402434,Tomat


In [4]:
# memisahkan data ke train dan test
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [5]:
# memisahkan label
train_features = train_dataset.copy()
test_features = test_dataset.copy()
# pd.get_dummies() untuk membuat one-hot encoding
train_labels = pd.get_dummies(train_features.pop('Label'))
test_labels = pd.get_dummies(test_features.pop('Label'))

In [19]:
print(test_labels)

    Cocor Bebek  Jambu  Kacang Panjang  Singkong  Tomat
9             1      0               0         0      0
12            1      0               0         0      0
21            0      1               0         0      0
25            0      1               0         0      0
36            0      1               0         0      0
37            0      1               0         0      0
39            0      1               0         0      0
44            0      0               1         0      0
46            0      0               1         0      0
47            0      0               1         0      0
58            0      0               1         0      0
64            0      0               0         1      0
65            0      0               0         1      0
67            0      0               0         1      0
70            0      0               0         1      0
81            0      0               0         0      1
83            0      0               0         0

In [6]:
# membuat neural network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np

# membuat normalization layer
normalizer = preprocessing.Normalization()

normalizer.adapt(np.array(train_features))

In [7]:
first = np.array(train_features[:1])
# mengecek apak normalization layer sudah berfungsi
with np.printoptions(precision=2, suppress=True):
    print('First example:',first)
    print()
    print('Normalized:', normalizer(first).numpy())

First example: [[3843.      0.78  846.89    0.07   41.99   85.28   22.97]]

Normalized: [[ 0.88 -0.2   2.27 -1.38 -1.12  0.66 -0.62]]


In [8]:
model = keras.models.Sequential()

model.add(normalizer)
model.add(layers.Dense(14, input_dim=7, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
history = model.fit(x=train_features, y=train_labels, batch_size=5, epochs=80, shuffle=True, validation_split=0.2)

epochs = history.epoch
hist = pd.DataFrame(history.history)

Epoch 1/80
13/13 [==============================] - 0s 15ms/step - loss: 1.6146 - accuracy: 0.3281 - val_loss: 1.9780 - val_accuracy: 0.1250
Epoch 2/80
13/13 [==============================] - 0s 2ms/step - loss: 1.5616 - accuracy: 0.3281 - val_loss: 1.9421 - val_accuracy: 0.1875
Epoch 3/80
13/13 [==============================] - 0s 3ms/step - loss: 1.5137 - accuracy: 0.3438 - val_loss: 1.9029 - val_accuracy: 0.1875
Epoch 4/80
13/13 [==============================] - 0s 3ms/step - loss: 1.4671 - accuracy: 0.3438 - val_loss: 1.8695 - val_accuracy: 0.1875
Epoch 5/80
13/13 [==============================] - 0s 3ms/step - loss: 1.4260 - accuracy: 0.3438 - val_loss: 1.8365 - val_accuracy: 0.2500
Epoch 6/80
13/13 [==============================] - 0s 3ms/step - loss: 1.3815 - accuracy: 0.3750 - val_loss: 1.8042 - val_accuracy: 0.2500
Epoch 7/80
13/13 [==============================] - 0s 3ms/step - loss: 1.3431 - accuracy: 0.3750 - val_loss: 1.7783 - val_accuracy: 0.2500
Epoch 8/80
13/13 [=

13/13 [==============================] - 0s 3ms/step - loss: 0.3966 - accuracy: 0.9531 - val_loss: 1.0690 - val_accuracy: 0.8125
Epoch 60/80
13/13 [==============================] - 0s 2ms/step - loss: 0.3882 - accuracy: 0.9531 - val_loss: 1.0648 - val_accuracy: 0.8125
Epoch 61/80
13/13 [==============================] - 0s 2ms/step - loss: 0.3797 - accuracy: 0.9531 - val_loss: 1.0618 - val_accuracy: 0.8125
Epoch 62/80
13/13 [==============================] - 0s 2ms/step - loss: 0.3727 - accuracy: 0.9531 - val_loss: 1.0587 - val_accuracy: 0.8125
Epoch 63/80
13/13 [==============================] - 0s 2ms/step - loss: 0.3650 - accuracy: 0.9531 - val_loss: 1.0568 - val_accuracy: 0.8125
Epoch 64/80
13/13 [==============================] - 0s 2ms/step - loss: 0.3569 - accuracy: 0.9531 - val_loss: 1.0546 - val_accuracy: 0.8125
Epoch 65/80
13/13 [==============================] - 0s 2ms/step - loss: 0.3495 - accuracy: 0.9531 - val_loss: 1.0510 - val_accuracy: 0.8125
Epoch 66/80
13/13 [======

In [10]:
print("Evaluate on test data")
results = model.evaluate(test_features, test_labels, batch_size=4)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 0s 1ms/step - loss: 0.5056 - accuracy: 0.9500
test loss, test acc: [0.5055637359619141, 0.949999988079071]


In [34]:
prediction = model.predict([2592, 0.587480, 648.558, 0.077437, 62.147971, 98.496181, 34.956567])

In [23]:
print(np.argmax(prediction[0]))

1


In [35]:
print(prediction)

[[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [9.9974889e-01 1.2922240e-05 6.1844061e-05 1.7642787e-04 2.6401832e-12]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [3.6495037e-03 1.4389096e-06 1.0276807e-01 8.9357835e-01 2.7010733e-06]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]


In [36]:
cocor = 0
jambu = 0
kacang = 0
singkong = 0
tomat = 0

for i in range(7):
    for j in range(5):
        if j == 0:
            cocor += prediction[i][j]
        elif j == 1:
            jambu += prediction[i][j]
        elif j == 2:
            kacang += prediction[i][j]
        elif j == 3:
            singkong += prediction[i][j]
        elif j == 4:
            tomat += prediction[i][j]

In [37]:
print(cocor)
print(jambu)
print(kacang)
print(singkong)
print(tomat)

1.0033983893226832
5.0000143611495105
0.1028299150566454
0.8937547784153139
2.7010759394326123e-06
